In [ ]:
import numpy as np

from scipy.stats import norm, chi2, t

import matplotlib.pyplot as plt
%matplotlib inline

# Distribuzione $t$ di Student

Estraiamo $N$ valori da una distribuzione **gaussiana** con valore medio vero $\mu$ e deviazione standard $\sigma$.

Calcoliamo *dai dati* il valor medio e la deviazione standard **stimati**:
$$
\overline{z}=\frac{1}{N}\sum_{i=1}^N z_i, \qquad\quad s_z=\sqrt{\frac{1}{N-1}\sum_{i=1}^N(z_i-\overline{z})^2}
$$

La deviazione standard **stimata** della media $\overline{z}$ è
$$
s_{\overline{z}}=\frac{s_z}{\sqrt{N}}.
$$

Con $\overline{z}$ e $s_{\overline{z}}$ costruiamo una nuova variabile
$$
t=\frac{\overline{z}-\mu}{s_{\overline{z}}}=\frac{\overline{z}-\mu}{\frac{s}{\sqrt{N}}}.
$$
Esaminiamo empiricamente come si distribuisce la variabile casuale $t$ costruita estrando $Rep=5000$ volte sequenze di 5 variabili gaussiane $z$. Le $z$ sono estratte da una gaussiana di media vera $\mu=5$.

In [ ]:
Rep = 5000
N = 5
mu = 5

In [ ]:
zvalues = np.loadtxt('./data/zvalues.csv',delimiter=',')

In [ ]:
zbar = zvalues.mean(axis=1)
s_z = zvalues.std(axis=1,ddof=1)

In [ ]:
s_zbar = s_z/np.sqrt(N)

In [ ]:
t_values = (zbar-mu)/s_zbar

Costruiamo l'istogramma dei valori di $t$

In [ ]:
width = 0.5
bins = np.arange(-4.5,5.,width)

In [ ]:
Oi, bins = np.histogram(t_values,bins=bins)
print(Oi)

In [ ]:
center_bins = np.sum(np.vstack([bins[:-1],bins[1:]]),axis=0)/2.
print(center_bins)

In [ ]:
fig,ax = plt.subplots()

ax.bar(center_bins,Oi,width=0.5,alpha=0.6,ec='k',lw=0.5)

Con un test del $\chi^2$ valutiamo se la distribuzione empirica di $t$ è **gaussiana**.

In [ ]:
tran = np.arange(-5,5,.05)

In [ ]:
cdfs = norm.cdf(bins)
Ei = np.diff(cdfs)*Rep

In [ ]:
fig,ax = plt.subplots()

ax.bar(center_bins,Oi,width=0.5,alpha=0.6,ec='k',lw=0.5)
ax.scatter(center_bins,Ei,c='k')
ax.plot(tran,Rep*width*norm.pdf(tran),c='k',lw=1,label='Gauss')

ax.legend(loc='upper right')

In [ ]:
chisq = np.sum((Oi-Ei)**2/Ei)
print(chisq)

In [ ]:
1-chi2.cdf(chisq,len(Oi)-2)

La variabile casuale $t$ **non è** distribuita come una gaussiana.

La distribuzione seguita da $t$ è quella di **Student**
$$
\textup{PDF}(t;D)= T_D\left(1+\frac{t^2}{D}\right)^{-\frac{D+1}{2}},
$$
dove $D$ è il numero di **gradi di libertà** della distribuzione di $t$.

Nel nostro caso: $D=N-1$ perché abbiamo ricavato dai dati il parametro $s_z$.

In [ ]:
cdfs_t = t.cdf(bins,N-1)
Ei_t = np.diff(cdfs_t)*Rep

In [ ]:
fig,ax = plt.subplots()

ax.bar(center_bins,Oi,width=0.5,alpha=0.6,ec='k',lw=0.5)

ax.plot(tran,Rep*width*norm.pdf(tran),c='k',label='Gauss')

ax.scatter(center_bins,Ei_t,c='r')
ax.plot(tran,Rep*width*t.pdf(tran,N-1),c='r',label='Student')

ax.legend(loc='upper right')

In [ ]:
chisq_t = np.sum((Oi-Ei_t)**2/Ei_t)
print(chisq_t)

In [ ]:
1-chi2.cdf(chisq_t,len(Oi)-3)

La distribuzione di $t$ varia al variare dei **gradi di libertà**

In [ ]:
Ran = [1,2,3,5,6,7,10,12,15,20, 25, 30]
Nmax = len(Ran)
chiran = np.arange(0,30,0.05)
colors = plt.cm.Reds(np.linspace(0.2,1,Nmax))

In [ ]:
fig,ax = plt.subplots()

ax.set_ylim(-.03,.45)
ax.set_xlabel('$t$',fontsize=12)
ax.set_ylabel('PDF',fontsize=12)

for i,n in enumerate(Ran):
    ax.plot(tran,t.pdf(tran,n),c=colors[i],lw=2,label='{}'.format(n))

ax.plot(tran,norm.pdf(tran),c='blue',lw=3,label='Gauss')


ax.axhline(0,c='k',lw=.5)
ax.axvline(0,c='k',lw=0.5)
ax.legend(loc='upper right',title='$n$')

# Il test di Student

Immaginiamo di aver raccolto dei dati su un nuovo dispositivo che, montato sul motore di un'automobile, ne migliora il consumo. 

Facciamo delle misure su $N=7$ automobili **prima** dell'installazione (gruppo di controllo) e **dopo** l'installazione. 

In [ ]:
consumi1,consumi2 = np.loadtxt('./data/consumi.csv',delimiter=',',skiprows=1,unpack=True)
N=len(consumi1)

Possiamo valutare la differenza di consumi, automobile per automobile, fra dopo e prima l'installazione:

In [ ]:
diff = consumi2-consumi1
diff_bar = diff.mean()
s_diff = diff.std(ddof=1)
s_diff_bar = s_diff/np.sqrt(N)

Facciamo l'**ipotesi nulla** che il dispostivo **non modifichi** effettivamente i consumi, ma che le misurazioni fra prima e dopo differiscano solamente per fluttuazioni statistiche. 

Secondo l'ipotesi nulla, la **media vera** delle differenze deve quindi essere zero, $\mu=0$. Costruiamo la variabile $t$ di Student:
$$
t_O = \frac{\overline{d}-\mu}{s_{\overline{d}}}=\frac{\overline{d}}{s_{\overline{d}}}
$$

In [ ]:
tO = (diff_bar-0)/s_diff_bar
print(tO)

Se il dispositivo ha un impatto sui consumi, allora $\overline{d}>0$ e anche $t_O>0$.

Possiamo chiederci qual è la probabilità che $t\geq t_O$:
\begin{align}
P(t\geq t_O)&=\int_{t_O}^{+\infty}\textup{PDF}(t;\textup{DOF})\,dt\\
&=1-\int_{-\infty}^{t_O}\textup{PDF}(t;\textup{DOF})\,dt\\
&=1-\textup{CDF}(t_O;\textup{DOF})
\end{align}

- Se $P(t>t_O)>\alpha$ allora **non possiamo rigettare** l'ipotesi nulla.
- Se $P(t>t_O)<\alpha$ allora **rigettiamo** l'ipotesi nulla al livello di significatività $\alpha$.

In [ ]:
tran = np.arange(-5,5,.01)

In [ ]:
fig,ax = plt.subplots()

ax.set_xlabel('t',fontsize=12)
ax.set_ylabel('PDF',fontsize=12)

ax.plot(tran,t.pdf(tran,N-1),c='r',label=r'$\mathrm{PDF}(t;6)$')

ax.fill_between(tran[tran>=tO],t.pdf(tran[tran>=tO],N-1),fc='r',alpha=0.3)
ax.plot([tO,tO],[0.,t.pdf(tO,N-1)],c='k')

ax.axhline(0,c='k',lw=0.5)
ax.axvline(0,c='k',lw=0.5)

ax.annotate(r"$P(t\geq t_O)$",(2.5,0.01),(5.,0.15),ha='right',arrowprops=dict(arrowstyle="-"),fontsize=12)
ax.annotate(r"$t_O$",(1.8,0.015))

ax.legend(loc='upper right')

In [ ]:
1-t.cdf(tO,N-1)

Se fissiamo $\alpha=5\%$, $P(t>t_O)<\alpha=0.05$: **rigettiamo** l'ipotesi nulla (nessuna efficacia del dispositivo) al livello di significatività del 5%.<br>
La differenza è significativa e **non è dovuta al caso**. Il dispositivo aumenta la resa ($d > 0$).

# Accelerazione di gravità

In un esperimento si misura $N=5$ volte il periodo di piccole oscillazione in s di un pendolo semplice lungo $l=2.000\pm0.001$ m.<br>
I dati sono raccolti nel file `./data/periodo_pendolo.dat`.

Usando la formula del periodo del pendolo semplice per piccole oscillazioni si vuole misurare l'accelerazione di gravità $g$. Dalle misure si può ricavare come miglior *stimatore* del periodo, il suo valore medio $\overline{T}$ e calcolare
$$
g=4\pi^2\frac{l}{\overline{T}}
$$

La misura di $g$ ottenuta, è **compatibile** con il "valore vero"  $g^0=9.80665$ m/s$^2$?

1) Ottenere *dai dati* il valore medio $\overline{T}$ e la deviazione standard **stimata** sul valore medio $s_{\overline{T}}$.
2) Calcolare il valore di $g$ usando $\overline{T}$ e il suo errore $s_g$ **propagando** l'errore da $s_{\overline{T}}$ e $\sigma_l$.
3) Assumere l'**ipotesi nulla**: $g-g^0=0$.
4) Il numero di misurazioni di $T$ è piccolo: $N=5$. Costruire la variabile di Student osservata
   $$t_O=\frac{g-g^0}{s_g}$$
6) Il valore misurato di $g$ può in principio essere **più grande** o **più piccolo** del valore vero: dobbiamo valutare la probabilità che $$P(|t|\geq|t_O|)=P(t\leq-|t_O|)+P(t\geq|t_O|)$$
7) Usando $N-1$ gradi di libertà, cosa possiamo concludere dal test?